# ejabT1E Test Suite Overview

The file `Package/tests/test_ejab.R` contains 8 sequential tests that validate each function in the package. Tests use `stopifnot()` assertions -- any failure halts execution immediately.

A shared simulated dataset (N=500, seed=42) is created at Test 3 and reused through Tests 4-8, so the tests also verify that functions compose correctly in sequence.

---
## Test 1: `ejab01` -- Basic Computation

Computes a single eJAB01 value for a z-test (q=1) with n=100, p=0.05.

**Checks:**
- Output is finite
- Output is positive (Bayes factors are always > 0)

**Expected value (manual):**
$$\sqrt{100} \cdot \exp\left(-0.5 \cdot \frac{99}{100} \cdot \chi^2_{1, 0.95}\right) = 10 \cdot \exp(-0.5 \cdot 0.99 \cdot 3.8415)$$

In [ ]:
val <- ejab01(0.05, 100, 1)
stopifnot(is.finite(val), val > 0)

---
## Test 2: `ejab01` -- Vectorized Input

Passes vectors of length 3 with different p, n, and q values (including q=2 for a multi-dimensional test).

**Checks:**
- Output has correct length (3)
- All values are finite and positive

In [ ]:
vals <- ejab01(c(0.01, 0.03, 0.05), c(50, 100, 200), c(1, 1, 2))
stopifnot(length(vals) == 3, all(is.finite(vals)), all(vals > 0))

---
## Test 3: `objective_C` -- Finite Positive Output

Creates the shared simulation dataset:
- N = 500 results, seed = 42
- 20% are nulls (p ~ Unif(0, 0.05)), 80% are true effects (p ~ Unif(0, 0.01))
- All n = 50, q = 1

Evaluates the objective at C = 1.0.

**Checks:**
- Output is finite
- Output is non-negative (it's an integrated squared deviation)

In [ ]:
set.seed(42)
N <- 500
is_null <- sample(c(TRUE, FALSE), N, replace = TRUE, prob = c(0.2, 0.8))
p_sim <- ifelse(is_null, runif(N, 0, 0.05), runif(N, 0, 0.01))
n_sim <- rep(50L, N)
q_sim <- rep(1, N)
ejab_sim <- ejab01(p_sim, n_sim, q_sim)

obj_val <- objective_C(1.0, p_sim, ejab_sim, up = 0.05)
stopifnot(is.finite(obj_val), obj_val >= 0)

---
## Test 4: `estimate_Cstar` -- Grid Search Optimality

Runs the grid search on the simulated data.

**Checks:**
- C* is within the grid bounds [1/3, 3]
- The objective at C* is <= the objective at both grid endpoints (confirming it's a proper minimum, not an edge solution by coincidence)

In [ ]:
fit <- estimate_Cstar(p_sim, ejab_sim, up = 0.05)
stopifnot(fit$Cstar >= 1/3, fit$Cstar <= 3)
stopifnot(fit$objective <= objective_C(1/3, p_sim, ejab_sim, 0.05))
stopifnot(fit$objective <= objective_C(3, p_sim, ejab_sim, 0.05))

---
## Test 5: `detect_type1` -- Contradiction Logic

Detects candidate T1Es using the estimated C*.

**Checks:**
- All flagged results have p <= 0.05 (NHST significant)
- All flagged results have eJAB > C* (Bayes supports H0)

These are the two defining conditions of a Bayes/NHST contradiction.

In [ ]:
idx <- detect_type1(p_sim, ejab_sim, alpha = 0.05, Cstar = fit$Cstar)
stopifnot(all(p_sim[idx] <= 0.05))
stopifnot(all(ejab_sim[idx] > fit$Cstar))

---
## Test 6: `diagnostic_U` -- Finite Output

Computes U_i values for the detected candidates.

**Checks:**
- All U_i values are finite (no NaN/Inf from the d_i computation)

Note: The test does *not* assert U in [0,1] because with simulated (not purely null) data, some U_i may fall outside [0,1]. The [0,1] property holds under the assumption that all candidates are true T1Es.

In [ ]:
if (length(idx) > 0) {
  U <- diagnostic_U(p_sim[idx], n_sim[idx], q_sim[idx], alpha = 0.05, Cstar = fit$Cstar)
  stopifnot(all(is.finite(U)))
}

---
## Test 7: `posterior_t1e` -- Valid Probabilities

Computes posterior P(T1E) for each candidate.

**Checks:**
- All posterior probabilities are in [0, 1]

This verifies the normalization is correct and the summation over qualifying grid points doesn't exceed 1.

In [ ]:
if (length(idx) > 0) {
  post <- posterior_t1e(p_sim[idx], ejab_sim[idx], alpha = 0.05,
                        fit$all_objectives, fit$grid)
  stopifnot(all(post >= 0 & post <= 1))
}

---
## Test 8: `ejab_pipeline` -- End-to-End Consistency

Runs the full pipeline wrapper on the same data and verifies it produces identical results to the individual function calls.

**Checks:**
- `result$Cstar` is not NULL
- `result$Cstar` matches the C* from the standalone `estimate_Cstar` call (Test 4)

This confirms the pipeline correctly composes all steps and doesn't introduce any discrepancy.

In [ ]:
df_test <- data.frame(ID = seq_len(N), p = p_sim, n = n_sim, q = q_sim)
result <- ejab_pipeline(df_test, up = 0.05, alpha = 0.05, plot = FALSE)
stopifnot(!is.null(result$Cstar))
stopifnot(result$Cstar == fit$Cstar)

---
## Test Coverage Summary

| Function | Tests | What's Validated |
|----------|-------|------------------|
| `ejab01` | 1, 2 | Scalar + vector input, positivity, finiteness |
| `objective_C` | 3 | Non-negative, finite output |
| `estimate_Cstar` | 4 | In-bounds C*, proper minimum |
| `detect_type1` | 5 | Contradiction conditions hold for all flagged |
| `diagnostic_U` | 6 | Finite output (no division-by-zero) |
| `posterior_t1e` | 7 | Valid probabilities in [0, 1] |
| `ejab_pipeline` | 8 | Consistent with individual calls |

**Not tested** (would require targeted simulation):
- Input validation / error messages (e.g., n=1, p=0, alpha > up)
- `diagnostic_qqplot` (graphical output)
- Calibration of U_i ~ Unif(0,1) under true null (requires large pure-null simulation)